In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/keypoints_final_256.csv')
df_test = pd.read_csv('/content/drive/MyDrive/keypoints_final_test_256.csv')

In [ ]:
df = df.drop('Unnamed: 0',1)
df_test = df_test.drop('Unnamed: 0',1)

<ipython-input-185-2408e182c2cf>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop('Unnamed: 0',1)
<ipython-input-185-2408e182c2cf>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df_test = df_test.drop('Unnamed: 0',1)


In [ ]:
df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,1.1
0,0.561012,0.302157,-0.123826,0.564638,0.289460,-0.105465,0.567299,0.289761,-0.105632,0.569772,...,0.533098,0.883884,0.116445,0.566635,0.933335,0.036987,0.536138,0.938561,-0.000178,tadasana
1,0.561800,0.300378,-0.112502,0.564178,0.286860,-0.094147,0.566694,0.287539,-0.094309,0.568935,...,0.532270,0.883887,0.118631,0.570173,0.931963,0.035905,0.532538,0.933740,-0.001898,tadasana
2,0.561128,0.303233,-0.398081,0.564855,0.286085,-0.388905,0.567303,0.285346,-0.389124,0.569531,...,0.527861,0.878289,0.133518,0.566152,0.918341,0.058699,0.534654,0.928207,-0.001211,tadasana
3,0.560880,0.309663,-0.300727,0.564582,0.291333,-0.289611,0.567254,0.290768,-0.289883,0.570002,...,0.536183,0.912010,0.188811,0.567818,0.914286,0.152388,0.530323,0.931029,0.077729,tadasana
4,0.561824,0.313649,-0.342597,0.565599,0.294846,-0.333475,0.568605,0.294382,-0.333750,0.571083,...,0.540910,0.887583,0.192771,0.571397,0.923574,0.116451,0.531012,0.930395,0.070042,tadasana
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13124,0.486648,0.286260,-0.089110,0.490879,0.273558,-0.067470,0.493832,0.272559,-0.067595,0.497064,...,0.470475,0.872472,0.102894,0.496999,0.912968,0.022522,0.466042,0.915865,-0.025367,tadasana
13125,0.482443,0.300422,-0.362747,0.487832,0.286010,-0.350695,0.491121,0.285521,-0.350950,0.493022,...,0.474411,0.877378,0.126974,0.494784,0.912487,0.072589,0.465412,0.914555,0.006590,tadasana
13126,0.480868,0.294958,-0.321305,0.485442,0.280226,-0.312262,0.488200,0.280182,-0.312505,0.491096,...,0.473303,0.876180,0.188003,0.495985,0.914854,0.099370,0.465068,0.916696,0.075990,tadasana
13127,0.476346,0.293068,-0.347669,0.480525,0.276444,-0.338560,0.483666,0.276016,-0.338818,0.485902,...,0.473899,0.877159,0.200288,0.494451,0.915021,0.085202,0.464427,0.913910,0.084059,tadasana


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import LabelEncoder

In [ ]:
lbe = LabelEncoder()
df['1.1']= lbe.fit_transform(df['1.1'])
df_test['1.1']= lbe.fit_transform(df_test['1.1'])
dict(zip(lbe.classes_, lbe.transform(lbe.classes_)))

{'bhujan': 0, 'padmasan': 1, 'shav': 2, 'tadasana': 3, 'trik': 4, 'vriksh': 5}

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
class YogaDataset(Dataset):
  def __init__(self, df):
    self.features = df.drop(['1.1'], axis=1)
    self.target = df['1.1']

  def __len__(self):
    return len(self.features)
  
  def __getitem__(self, index):
    features = self.features.loc[index]
    target = self.target[index]
    return torch.tensor(features.tolist()).float().to(device), torch.tensor(target).long().to(device)

In [ ]:
# df_feat = df.drop(['1.1'], axis=1)

In [ ]:
device

'cpu'

In [ ]:
# target = df['1.1']

In [ ]:
data = YogaDataset(df)

In [ ]:
feat, target = data[100]

In [ ]:
target

tensor(3)

In [ ]:
class RNN(nn.Module):
  def __init__(self,hidden_dim=20, input_dim=99, sequence_num=16, n_layers=1):
    super(RNN, self).__init__()
    self.input_dim = input_dim
    self.sequence_num = sequence_num
    self.n_layers = n_layers
    self.hidden_dim = hidden_dim
    self.rnn = nn.RNN(input_dim,hidden_dim, n_layers, batch_first=True)
    self.fc = nn.Linear(hidden_dim, 6)
  def forward(self, input):
    h0 = torch.zeros(self.n_layers,  self.hidden_dim)
    out, _ = self.rnn(input,h0)
    pred = self.fc(out)
    output = nn.Softmax( dim=1)(pred)
    return output

In [ ]:
model = RNN().to(device)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
model

RNN(
  (rnn): RNN(99, 20, batch_first=True)
  (fc): Linear(in_features=20, out_features=6, bias=True)
)

In [ ]:
def get_data():
    train = YogaDataset(df)
    test = YogaDataset(df_test)
    trn_dl = DataLoader(train, batch_size=16,shuffle=True, drop_last=True)
    test_dl = DataLoader(test, batch_size=16, shuffle=True, drop_last=True)
    return trn_dl, test_dl

In [ ]:
trn_ldr, test_ldr = get_data()
# for x in trn_ldr:
#   print(x)
#   break

In [ ]:
def train_batch(x, y, model, loss_fn, opt):
    model.train()
    prediction = model(x)
    # print(y.shape)
    # print(y)
    # print(prediction.shape)
    batch_loss = loss_fn(prediction, y)
    batch_loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    return batch_loss.item()

In [ ]:
@torch.no_grad()
def accuracy(x, y, model):
    model.eval()
    pred = model(x)
    _, is_correct = torch.max(pred.data,1)
    acc_all = (is_correct==y).detach().cpu().numpy()
    return acc_all

In [ ]:
@torch.no_grad()
def val_loss_trn(x, y, model, loss_fn):
  prediction = model(x)
  val_loss = loss_fn(prediction, y)
  return val_loss.item()

In [ ]:
train_loss = []
train_accuracies= []
val_loss = [] 
val_accuracies = []

for i in range(10):
    print(f'Epoch: _________*****{i}*****_______')
    train_epoch_losses, train_epoch_accuracies = [], []
    val_epoch_accuracies, val_epoch_losses = [], []



    for ix, batch in (enumerate(iter(trn_ldr))):

        x, y = batch
        train_epoch_losses.append(train_batch(x, y, model, loss_fn, optimizer))
    train_epoch_loss = np.array(train_epoch_losses).mean()
    print(f'Epoch: _________*****{i} Training Loss : {train_epoch_loss} *****_______')



    for ix, batch in (enumerate(iter(trn_ldr))):
        x, y = batch
        # x = x.permute(0, 3,1,2)
        train_epoch_accuracies.append(sum(accuracy(x, y, model)) / len(y))
    train_epoch_accuracy = np.array(train_epoch_accuracies).mean()
    print(f'Epoch: _________*****{i} Training Accuracy: {train_epoch_accuracy} *****_______')


    for ix, batch in (enumerate(iter(test_ldr))):
        x, y = batch
        # x = x.permute(0, 3,1,2)
        val_epoch_losses.append(val_loss_trn(x, y, model, loss_fn))
    val_epoch_loss = np.array(val_epoch_losses).mean()
    print(f'Epoch: _________*****{i} Validation Loss : {val_epoch_loss} *****_______')

    for ix, batch in (enumerate(iter(test_ldr))):
        x, y = batch
        # x = x.permute(0, 3,1,2)
        val_epoch_accuracies.append(sum(accuracy(x, y, model)) / len(y))
    val_epoch_accuracy = np.array(val_epoch_accuracies).mean()
    print(f'Epoch: _________*****{i} Validation Accuracy. {val_epoch_accuracy} *****_______')
    print('\n')

    train_loss.append(train_epoch_loss)
    train_accuracies.append(train_epoch_accuracy)
    val_accuracies.append(val_epoch_accuracy)
    val_loss.append(val_epoch_loss)

Epoch: _________*****0*****_______
Epoch: _________*****0 Training Loss : 1.3447024740823885 *****_______
Epoch: _________*****0 Training Accuracy: 0.9511432926829269 *****_______
Epoch: _________*****0 Validation Loss : 1.116986921440018 *****_______
Epoch: _________*****0 Validation Accuracy. 0.9653745644599303 *****_______


Epoch: _________*****1*****_______
Epoch: _________*****1 Training Loss : 1.1100339143741422 *****_______
Epoch: _________*****1 Training Accuracy: 0.965625 *****_______
Epoch: _________*****1 Validation Loss : 1.095366268623166 *****_______
Epoch: _________*****1 Validation Accuracy. 0.9577526132404182 *****_______


Epoch: _________*****2*****_______
Epoch: _________*****2 Training Loss : 1.0865559181062188 *****_______
Epoch: _________*****2 Training Accuracy: 0.9766768292682927 *****_______
Epoch: _________*****2 Validation Loss : 1.089306939769705 *****_______
Epoch: _________*****2 Validation Accuracy. 0.9570993031358885 *****_______


Epoch: _________****